In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
!pip install emoji
import emoji
!pip install Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('indonesian'))
import re
import json
import ast
import itertools
import seaborn as sns
import pytz
import string
import numpy as np
from wordcloud import WordCloud,STOPWORDS
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.1 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/data/Ulasan.csv')

In [ ]:
df.head()

,Source.Name,rating_star,author_username,comment
0,1.xlsx,5,M***a,"pas buat bekel, ini yg mini sih, tali panjangn..."
1,1.xlsx,5,R***i,tas ny bagus..simple tp msh keliatan mewah..ba...
2,1.xlsx,5,N***k,"Bagus, sesuai dengan deskripsi barang nya. Pak..."
3,1.xlsx,5,Sri,NaN
4,1.xlsx,5,Sri,NaN


In [ ]:
df.isnull().sum().sort_values(ascending=False)

comment            118
Source.Name          0
rating_star          0
author_username      0
dtype: int64

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.shape

(3243, 4)

## Text Cleaning

In [ ]:
# Funsgi untuk membersihakn teks
def clean_text(Text):
     # remove stock market tickers like $GE
    Text = re.sub(r'\$\w*', '',str(Text))
    # replace kata yang berulang-ulang ('oooooo' menjadi '00')
    Text = re.sub(r'(.)\1+', r'\1\1', Text)
    # Mengganti dua titik dengan spasi
    Text = re.sub(r'\.{2,}', ' ', Text)
    # Menghapus @username
    Text = re.sub('@[^\s]+','',Text)
    # remove old style retweet text "RT"
    Text = re.sub(r'^RT[\s]+', '', Text)
    #remove angka
    Text = re.sub('[0-9]+', '', Text)
    #remove url
    Text = re.sub(r"http\S+", "", Text)
    # remove hashtags
    Text = re.sub(r'#', '', Text)
    # Strip space, " and ' from tweet
    Text = Text.strip(' "\'')
    # Replace multiple spaces with a single space
    Text = re.sub(r'\s+', ' ', Text)
    #hapus tanda baca
    Text = Text.translate(str.maketrans("","",string.punctuation))

    return Text

In [ ]:
# Terapkan fungsi pembersihan pada DataFrame
df['cleaned_comment'] = df['comment'].apply(lambda x: clean_text(x))

In [ ]:
df

,Source.Name,rating_star,author_username,comment,cleaned_comment
0,1.xlsx,5,M***a,"pas buat bekel, ini yg mini sih, tali panjangn...",pas buat bekel ini yg mini sih tali panjangnya...
1,1.xlsx,5,R***i,tas ny bagus..simple tp msh keliatan mewah..ba...,tas ny bagus simple tp msh keliatan mewah baha...
2,1.xlsx,5,N***k,"Bagus, sesuai dengan deskripsi barang nya. Pak...",Bagus sesuai dengan deskripsi barang nya Paket...
5,1.xlsx,5,Sri,kreeeeeen👍👍,kreen👍👍
6,1.xlsx,3,Diah,kemahalan,kemahalan
...,...,...,...,...,...
3356,9.xlsx,5,F***H,"bahannya bagus ,gede juga ternyata xixixxi. ma...",bahannya bagus gede juga ternyata xixixxi maka...
3357,9.xlsx,5,rini,bahan tebal. ukuran besar\nkuat . \nwarna siip,bahan tebal ukuran besar kuat warna siip
3358,9.xlsx,5,Amien,tas nya besar bisa muat banyak,tas nya besar bisa muat banyak
3359,9.xlsx,5,P***i,"Bahannya tebel, muat untuk laptop 14 inch, unt...",Bahannya tebel muat untuk laptop inch untuk ha...


## Menghapus Emoji

In [ ]:
# Fungsi untuk menghapus emoji
def clean_tweet(text):
  # Mengubah teks ke lowercase
  text = text.lower()

  # Menghapus emoticon
  emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
  text = emoji_pattern.sub(r'', text)

  # Menghapus non ASCII character
  encoded_string = text.encode("ascii", "ignore")
  text = encoded_string.decode()

  return(text)

In [ ]:
df['cleaned_comment'] = df['cleaned_comment'].astype(str)
df['Case_Folding_&_Clean_Emoji'] = df['cleaned_comment'].apply(lambda x:clean_tweet(x))
df

,Source.Name,rating_star,author_username,comment,cleaned_comment,Case_Folding_&_Clean_Emoji
0,1.xlsx,5,M***a,"pas buat bekel, ini yg mini sih, tali panjangn...",pas buat bekel ini yg mini sih tali panjangnya...,pas buat bekel ini yg mini sih tali panjangnya...
1,1.xlsx,5,R***i,tas ny bagus..simple tp msh keliatan mewah..ba...,tas ny bagus simple tp msh keliatan mewah baha...,tas ny bagus simple tp msh keliatan mewah baha...
2,1.xlsx,5,N***k,"Bagus, sesuai dengan deskripsi barang nya. Pak...",Bagus sesuai dengan deskripsi barang nya Paket...,bagus sesuai dengan deskripsi barang nya paket...
5,1.xlsx,5,Sri,kreeeeeen👍👍,kreen👍👍,kreen
6,1.xlsx,3,Diah,kemahalan,kemahalan,kemahalan
...,...,...,...,...,...,...
3356,9.xlsx,5,F***H,"bahannya bagus ,gede juga ternyata xixixxi. ma...",bahannya bagus gede juga ternyata xixixxi maka...,bahannya bagus gede juga ternyata xixixxi maka...
3357,9.xlsx,5,rini,bahan tebal. ukuran besar\nkuat . \nwarna siip,bahan tebal ukuran besar kuat warna siip,bahan tebal ukuran besar kuat warna siip
3358,9.xlsx,5,Amien,tas nya besar bisa muat banyak,tas nya besar bisa muat banyak,tas nya besar bisa muat banyak
3359,9.xlsx,5,P***i,"Bahannya tebel, muat untuk laptop 14 inch, unt...",Bahannya tebel muat untuk laptop inch untuk ha...,bahannya tebel muat untuk laptop inch untuk ha...


## Mengganti Slang word (kata gaul)

In [ ]:
# Import data slang word
df_slang= pd.read_csv('/content/drive/MyDrive/data/colloquial-indonesian-lexicon.csv')
df_slang

,slang,formal,In-dictionary,context,category1,category2,category3
0,woww,wow,1,wow,elongasi,0,0
1,aminn,amin,1,Selamat ulang tahun kakak tulus semoga panjang...,elongasi,0,0
2,met,selamat,1,Met hari netaas kak!? Wish you all the best @t...,abreviasi,0,0
3,netaas,menetas,1,Met hari netaas kak!? Wish you all the best @t...,afiksasi,elongasi,0
4,keberpa,keberapa,0,Birthday yg keberpa kak?,abreviasi,0,0
...,...,...,...,...,...,...,...
15001,gataunya,enggak taunya,0,Ini kaya nenek2 ya beb gataunya agnezz @yugime...,akronim,0,0
15002,gtau,enggak tau,0,Stidaknya mrka may berkarya Dan berusaha yg tr...,akronim,abreviasi,0
15003,gatau,enggak tau,0,Ih gatau malu,akronim,0,0
15004,fans2,fan-fan,0,Jkt48 adalah tempat di mana sesama fans saling...,reduplikasi,naturalisasi,0


In [ ]:
# Proses Slang Word
def replace_slang(tweets):
    tweets = tweets.lower()
    res = ''
    for item in tweets.split():
        if item in df_slang.slang.values:
            res += df_slang[df_slang['slang'] == item]['formal'].iloc[0]
        else:
            res += item
        res += ' '
    return res

In [ ]:
df['slang_word'] = df['Case_Folding_&_Clean_Emoji'].apply(replace_slang)
df

,Source.Name,rating_star,author_username,comment,cleaned_comment,Case_Folding_&_Clean_Emoji,slang_word
0,1.xlsx,5,M***a,"pas buat bekel, ini yg mini sih, tali panjangn...",pas buat bekel ini yg mini sih tali panjangnya...,pas buat bekel ini yg mini sih tali panjangnya...,pas buat bekel ini yang mini sih tali panjangn...
1,1.xlsx,5,R***i,tas ny bagus..simple tp msh keliatan mewah..ba...,tas ny bagus simple tp msh keliatan mewah baha...,tas ny bagus simple tp msh keliatan mewah baha...,tas nya bagus simple tapi masih kelihatan mewa...
2,1.xlsx,5,N***k,"Bagus, sesuai dengan deskripsi barang nya. Pak...",Bagus sesuai dengan deskripsi barang nya Paket...,bagus sesuai dengan deskripsi barang nya paket...,bagus sesuai dengan deskripsi barang nya paket...
5,1.xlsx,5,Sri,kreeeeeen👍👍,kreen👍👍,kreen,keren
6,1.xlsx,3,Diah,kemahalan,kemahalan,kemahalan,kemahalan
...,...,...,...,...,...,...,...
3356,9.xlsx,5,F***H,"bahannya bagus ,gede juga ternyata xixixxi. ma...",bahannya bagus gede juga ternyata xixixxi maka...,bahannya bagus gede juga ternyata xixixxi maka...,bahannya bagus gede juga ternyata xixixxi maka...
3357,9.xlsx,5,rini,bahan tebal. ukuran besar\nkuat . \nwarna siip,bahan tebal ukuran besar kuat warna siip,bahan tebal ukuran besar kuat warna siip,bahan tebal ukuran besar kuat warna sip
3358,9.xlsx,5,Amien,tas nya besar bisa muat banyak,tas nya besar bisa muat banyak,tas nya besar bisa muat banyak,tas nya besar bisa muat banyak
3359,9.xlsx,5,P***i,"Bahannya tebel, muat untuk laptop 14 inch, unt...",Bahannya tebel muat untuk laptop inch untuk ha...,bahannya tebel muat untuk laptop inch untuk ha...,bahannya tebal muat untuk laptop inch untuk ha...


## Tokenizing

In [ ]:
#NLTK word tokenize
def word_tokenize_wrapper(Tweets):
 return word_tokenize(Tweets)
df['Tokenizing'] = df['slang_word'].apply(word_tokenize_wrapper)
df

,Source.Name,rating_star,author_username,comment,cleaned_comment,Case_Folding_&_Clean_Emoji,slang_word,Tokenizing
0,1.xlsx,5,M***a,"pas buat bekel, ini yg mini sih, tali panjangn...",pas buat bekel ini yg mini sih tali panjangnya...,pas buat bekel ini yg mini sih tali panjangnya...,pas buat bekel ini yang mini sih tali panjangn...,"[pas, buat, bekel, ini, yang, mini, sih, tali,..."
1,1.xlsx,5,R***i,tas ny bagus..simple tp msh keliatan mewah..ba...,tas ny bagus simple tp msh keliatan mewah baha...,tas ny bagus simple tp msh keliatan mewah baha...,tas nya bagus simple tapi masih kelihatan mewa...,"[tas, nya, bagus, simple, tapi, masih, kelihat..."
2,1.xlsx,5,N***k,"Bagus, sesuai dengan deskripsi barang nya. Pak...",Bagus sesuai dengan deskripsi barang nya Paket...,bagus sesuai dengan deskripsi barang nya paket...,bagus sesuai dengan deskripsi barang nya paket...,"[bagus, sesuai, dengan, deskripsi, barang, nya..."
5,1.xlsx,5,Sri,kreeeeeen👍👍,kreen👍👍,kreen,keren,[keren]
6,1.xlsx,3,Diah,kemahalan,kemahalan,kemahalan,kemahalan,[kemahalan]
...,...,...,...,...,...,...,...,...
3356,9.xlsx,5,F***H,"bahannya bagus ,gede juga ternyata xixixxi. ma...",bahannya bagus gede juga ternyata xixixxi maka...,bahannya bagus gede juga ternyata xixixxi maka...,bahannya bagus gede juga ternyata xixixxi maka...,"[bahannya, bagus, gede, juga, ternyata, xixixx..."
3357,9.xlsx,5,rini,bahan tebal. ukuran besar\nkuat . \nwarna siip,bahan tebal ukuran besar kuat warna siip,bahan tebal ukuran besar kuat warna siip,bahan tebal ukuran besar kuat warna sip,"[bahan, tebal, ukuran, besar, kuat, warna, sip]"
3358,9.xlsx,5,Amien,tas nya besar bisa muat banyak,tas nya besar bisa muat banyak,tas nya besar bisa muat banyak,tas nya besar bisa muat banyak,"[tas, nya, besar, bisa, muat, banyak]"
3359,9.xlsx,5,P***i,"Bahannya tebel, muat untuk laptop 14 inch, unt...",Bahannya tebel muat untuk laptop inch untuk ha...,bahannya tebel muat untuk laptop inch untuk ha...,bahannya tebal muat untuk laptop inch untuk ha...,"[bahannya, tebal, muat, untuk, laptop, inch, u..."


## Stopword

In [ ]:
#Proses Stopword Removal
def stopword_removal(Tweets):
    filtering = stopwords.words('indonesian','english')
    x = []
    data = []
    def myFunc(x):
        if x in filtering:
            return False
        else:
            return True
    fit = filter(myFunc, Tweets)
    for x in fit:
        data.append(x)
    return data
df['Filtering'] = df['Tokenizing'].apply(stopword_removal)
df

,Source.Name,rating_star,author_username,comment,cleaned_comment,Case_Folding_&_Clean_Emoji,slang_word,Tokenizing,Filtering
0,1.xlsx,5,M***a,"pas buat bekel, ini yg mini sih, tali panjangn...",pas buat bekel ini yg mini sih tali panjangnya...,pas buat bekel ini yg mini sih tali panjangnya...,pas buat bekel ini yang mini sih tali panjangn...,"[pas, buat, bekel, ini, yang, mini, sih, tali,...","[pas, bekel, mini, sih, tali, panjangnya, bera..."
1,1.xlsx,5,R***i,tas ny bagus..simple tp msh keliatan mewah..ba...,tas ny bagus simple tp msh keliatan mewah baha...,tas ny bagus simple tp msh keliatan mewah baha...,tas nya bagus simple tapi masih kelihatan mewa...,"[tas, nya, bagus, simple, tapi, masih, kelihat...","[tas, nya, bagus, simple, mewah, bahan, bagus,..."
2,1.xlsx,5,N***k,"Bagus, sesuai dengan deskripsi barang nya. Pak...",Bagus sesuai dengan deskripsi barang nya Paket...,bagus sesuai dengan deskripsi barang nya paket...,bagus sesuai dengan deskripsi barang nya paket...,"[bagus, sesuai, dengan, deskripsi, barang, nya...","[bagus, sesuai, deskripsi, barang, nya, paketn..."
5,1.xlsx,5,Sri,kreeeeeen👍👍,kreen👍👍,kreen,keren,[keren],[keren]
6,1.xlsx,3,Diah,kemahalan,kemahalan,kemahalan,kemahalan,[kemahalan],[kemahalan]
...,...,...,...,...,...,...,...,...,...
3356,9.xlsx,5,F***H,"bahannya bagus ,gede juga ternyata xixixxi. ma...",bahannya bagus gede juga ternyata xixixxi maka...,bahannya bagus gede juga ternyata xixixxi maka...,bahannya bagus gede juga ternyata xixixxi maka...,"[bahannya, bagus, gede, juga, ternyata, xixixx...","[bahannya, bagus, gede, xixixxi, makasii, sell..."
3357,9.xlsx,5,rini,bahan tebal. ukuran besar\nkuat . \nwarna siip,bahan tebal ukuran besar kuat warna siip,bahan tebal ukuran besar kuat warna siip,bahan tebal ukuran besar kuat warna sip,"[bahan, tebal, ukuran, besar, kuat, warna, sip]","[bahan, tebal, ukuran, kuat, warna, sip]"
3358,9.xlsx,5,Amien,tas nya besar bisa muat banyak,tas nya besar bisa muat banyak,tas nya besar bisa muat banyak,tas nya besar bisa muat banyak,"[tas, nya, besar, bisa, muat, banyak]","[tas, nya, muat]"
3359,9.xlsx,5,P***i,"Bahannya tebel, muat untuk laptop 14 inch, unt...",Bahannya tebel muat untuk laptop inch untuk ha...,bahannya tebel muat untuk laptop inch untuk ha...,bahannya tebal muat untuk laptop inch untuk ha...,"[bahannya, tebal, muat, untuk, laptop, inch, u...","[bahannya, tebal, muat, laptop, inch, harga, d..."


## Lemmatisasi dan ekstrak kata sifat

In [ ]:
# Install stanza
!pip install stanza
import stanza

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.1/990.1 kB 11.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-

In [ ]:
# Download model Bahasa Indonesia untuk Stanza
stanza.download('id')

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: id (Indonesian) ...


INFO:stanza:Downloaded file to /root/stanza_resources/id/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: id (Indonesian):
| Processor    | Package      |
-------------------------------
| tokenize     | gsd          |
| mwt          | gsd          |
| pos          | gsd_charlm   |
| lemma        | gsd_nocharlm |
| constituency | icon_charlm  |
| depparse     | gsd_charlm   |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: constituency
INFO:stanza:Loading: depparse
INFO:stanza:Done loading processors!


In [ ]:
nlp = stanza.Pipeline(lang='id', processors='tokenize,mwt,pos,lemma')

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: id (Indonesian):
| Processor | Package      |
----------------------------
| tokenize  | gsd          |
| mwt       | gsd          |
| pos       | gsd_charlm   |
| lemma     | gsd_nocharlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Done loading processors!


In [ ]:
# Fungsi untuk lemmatisasi
def lemmatize_text(text):
    text = ' '.join(text)  # Menggabungkan array kata-kata menjadi string
    doc = nlp(text)
    lemmatized_text = []
    for sent in doc.sentences:
        for word in sent.words:
            lemma = word.lemma if word.lemma != '' else word.text  # Menggunakan teks kata jika lemma kosong
            pos = word.upos if hasattr(word, 'upos') else ''  # Memeriksa apakah ada atribut 'upos'
            lemmatized_text.append((word.text, lemma, pos))
    return lemmatized_text

In [ ]:
# Fungsi untuk mengekstrak kata sifat
def extract_adjectives(lemmatized_text):
    adjectives = [lemma for _, lemma, pos in lemmatized_text if pos == 'ADJ']
    return adjectives

In [ ]:
# Menambahkan kolom baru 'Adjectives' untuk kata sifat
df['Adjectives'] = df['Filtering'].apply(lambda x: extract_adjectives(lemmatize_text(x)))

In [ ]:
df

,Source.Name,rating_star,author_username,comment,cleaned_comment,Case_Folding_&_Clean_Emoji,slang_word,Tokenizing,Filtering,Adjectives
0,1.xlsx,5,M***a,"pas buat bekel, ini yg mini sih, tali panjangn...",pas buat bekel ini yg mini sih tali panjangnya...,pas buat bekel ini yg mini sih tali panjangnya...,pas buat bekel ini yang mini sih tali panjangn...,"[pas, buat, bekel, ini, yang, mini, sih, tali,...","[pas, bekel, mini, sih, tali, panjangnya, bera...","[bagus, rigid, quality, tebal, bagus, biru, er..."
1,1.xlsx,5,R***i,tas ny bagus..simple tp msh keliatan mewah..ba...,tas ny bagus simple tp msh keliatan mewah baha...,tas ny bagus simple tp msh keliatan mewah baha...,tas nya bagus simple tapi masih kelihatan mewa...,"[tas, nya, bagus, simple, tapi, masih, kelihat...","[tas, nya, bagus, simple, mewah, bahan, bagus,...","[bagus, simple, mewah, bagus, tebal]"
2,1.xlsx,5,N***k,"Bagus, sesuai dengan deskripsi barang nya. Pak...",Bagus sesuai dengan deskripsi barang nya Paket...,bagus sesuai dengan deskripsi barang nya paket...,bagus sesuai dengan deskripsi barang nya paket...,"[bagus, sesuai, dengan, deskripsi, barang, nya...","[bagus, sesuai, deskripsi, barang, nya, paketn...","[bagus, rapi]"
5,1.xlsx,5,Sri,kreeeeeen👍👍,kreen👍👍,kreen,keren,[keren],[keren],[keren]
6,1.xlsx,3,Diah,kemahalan,kemahalan,kemahalan,kemahalan,[kemahalan],[kemahalan],[]
...,...,...,...,...,...,...,...,...,...,...
3356,9.xlsx,5,F***H,"bahannya bagus ,gede juga ternyata xixixxi. ma...",bahannya bagus gede juga ternyata xixixxi maka...,bahannya bagus gede juga ternyata xixixxi maka...,bahannya bagus gede juga ternyata xixixxi maka...,"[bahannya, bagus, gede, juga, ternyata, xixixx...","[bahannya, bagus, gede, xixixxi, makasii, sell...",[bagus]
3357,9.xlsx,5,rini,bahan tebal. ukuran besar\nkuat . \nwarna siip,bahan tebal ukuran besar kuat warna siip,bahan tebal ukuran besar kuat warna siip,bahan tebal ukuran besar kuat warna sip,"[bahan, tebal, ukuran, besar, kuat, warna, sip]","[bahan, tebal, ukuran, kuat, warna, sip]","[tebal, kuat]"
3358,9.xlsx,5,Amien,tas nya besar bisa muat banyak,tas nya besar bisa muat banyak,tas nya besar bisa muat banyak,tas nya besar bisa muat banyak,"[tas, nya, besar, bisa, muat, banyak]","[tas, nya, muat]",[]
3359,9.xlsx,5,P***i,"Bahannya tebel, muat untuk laptop 14 inch, unt...",Bahannya tebel muat untuk laptop inch untuk ha...,bahannya tebel muat untuk laptop inch untuk ha...,bahannya tebal muat untuk laptop inch untuk ha...,"[bahannya, tebal, muat, untuk, laptop, inch, u...","[bahannya, tebal, muat, laptop, inch, harga, d...","[tebal, bagus]"


In [ ]:
# Menggabungkan kata-kata menjadi satu kalimat tersusun dalam setiap baris
df['Adjectives'] = df['Adjectives'].apply(lambda x: ' '.join(x))

In [ ]:
df

,Source.Name,rating_star,author_username,comment,cleaned_comment,Case_Folding_&_Clean_Emoji,slang_word,Tokenizing,Filtering,Adjectives
0,1.xlsx,5,M***a,"pas buat bekel, ini yg mini sih, tali panjangn...",pas buat bekel ini yg mini sih tali panjangnya...,pas buat bekel ini yg mini sih tali panjangnya...,pas buat bekel ini yang mini sih tali panjangn...,"[pas, buat, bekel, ini, yang, mini, sih, tali,...","[pas, bekel, mini, sih, tali, panjangnya, bera...",bagus rigid quality tebal bagus biru ergonomic
1,1.xlsx,5,R***i,tas ny bagus..simple tp msh keliatan mewah..ba...,tas ny bagus simple tp msh keliatan mewah baha...,tas ny bagus simple tp msh keliatan mewah baha...,tas nya bagus simple tapi masih kelihatan mewa...,"[tas, nya, bagus, simple, tapi, masih, kelihat...","[tas, nya, bagus, simple, mewah, bahan, bagus,...",bagus simple mewah bagus tebal
2,1.xlsx,5,N***k,"Bagus, sesuai dengan deskripsi barang nya. Pak...",Bagus sesuai dengan deskripsi barang nya Paket...,bagus sesuai dengan deskripsi barang nya paket...,bagus sesuai dengan deskripsi barang nya paket...,"[bagus, sesuai, dengan, deskripsi, barang, nya...","[bagus, sesuai, deskripsi, barang, nya, paketn...",bagus rapi
5,1.xlsx,5,Sri,kreeeeeen👍👍,kreen👍👍,kreen,keren,[keren],[keren],keren
6,1.xlsx,3,Diah,kemahalan,kemahalan,kemahalan,kemahalan,[kemahalan],[kemahalan],
...,...,...,...,...,...,...,...,...,...,...
3356,9.xlsx,5,F***H,"bahannya bagus ,gede juga ternyata xixixxi. ma...",bahannya bagus gede juga ternyata xixixxi maka...,bahannya bagus gede juga ternyata xixixxi maka...,bahannya bagus gede juga ternyata xixixxi maka...,"[bahannya, bagus, gede, juga, ternyata, xixixx...","[bahannya, bagus, gede, xixixxi, makasii, sell...",bagus
3357,9.xlsx,5,rini,bahan tebal. ukuran besar\nkuat . \nwarna siip,bahan tebal ukuran besar kuat warna siip,bahan tebal ukuran besar kuat warna siip,bahan tebal ukuran besar kuat warna sip,"[bahan, tebal, ukuran, besar, kuat, warna, sip]","[bahan, tebal, ukuran, kuat, warna, sip]",tebal kuat
3358,9.xlsx,5,Amien,tas nya besar bisa muat banyak,tas nya besar bisa muat banyak,tas nya besar bisa muat banyak,tas nya besar bisa muat banyak,"[tas, nya, besar, bisa, muat, banyak]","[tas, nya, muat]",
3359,9.xlsx,5,P***i,"Bahannya tebel, muat untuk laptop 14 inch, unt...",Bahannya tebel muat untuk laptop inch untuk ha...,bahannya tebel muat untuk laptop inch untuk ha...,bahannya tebal muat untuk laptop inch untuk ha...,"[bahannya, tebal, muat, untuk, laptop, inch, u...","[bahannya, tebal, muat, laptop, inch, harga, d...",tebal bagus


In [ ]:
df.to_csv('cleaned_ulasan.csv')